In [1]:
import numpy as np
import warnings 
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data, meta = scipy.io.arff.loadarff('mtr-datasets/scm1d.arff')
data = pd.DataFrame(data)
data.head()

,timeunit,storageCost,interestRate,compidx0lt2,compidx0lt2l1,compidx0lt2l2,compidx0lt2l4,compidx0lt2l8,compidx1lt2,compidx2lt2,...,MTLp7,MTLp8,MTLp9,MTLp10,MTLp11,MTLp12,MTLp13,MTLp14,MTLp15,MTLp16
0,9.0,27.0,8.0,881.606,882.253,826.727,810.837,848.833,1468.412,815.832,...,2292.0,2181.0,1619.0,1849.0,1707.0,1932.0,2052.0,2409.0,2279.0,2138.0
1,10.0,27.0,8.0,869.455,881.606,882.253,802.480,770.544,1407.148,828.687,...,2276.0,2231.0,1649.0,1813.0,1696.0,1988.0,2140.0,2366.0,2307.0,2154.0
2,11.0,27.0,8.0,850.916,869.455,881.606,826.727,784.625,1424.312,837.975,...,2264.0,2203.0,1651.0,1739.0,1677.0,1991.0,2087.0,2295.0,2185.0,2131.0
3,12.0,27.0,8.0,862.066,850.916,869.455,882.253,792.403,1411.209,803.296,...,2279.0,2203.0,1627.0,1747.0,1669.0,1885.0,2072.0,2292.0,2187.0,2170.0
4,13.0,27.0,8.0,842.781,862.066,850.916,881.606,810.837,1428.685,813.118,...,2241.0,2198.0,1626.0,1740.0,1663.0,1874.0,2074.0,2257.0,2136.0,2164.0


In [3]:
X = data.iloc[:,:-16].values
y = data.iloc[:,-16:].values

scaler = StandardScaler()

X = scaler.fit_transform(X)
y = scaler.fit_transform(y)

print(X.shape,y.shape)

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

(9803, 280) (9803, 16)


In [4]:
print(X_train.shape,X_test.shape,y_train.shape,y_test.shape)

(7842, 280) (1961, 280) (7842, 16) (1961, 16)


In [5]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print('MAE - ',mean_absolute_error(y_train,y_test))
    print('RMSE - ',mean_squared_error(y_train,y_test)**0.5)
    print('R-squared - ',r2_score(y_train,y_test))
    
    r2 = r2_score(y_train,y_test)
    
    n = 9803
    p = 280

    adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    print('Adjusted R-squared - ',adj_r2)

In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout

Using TensorFlow backend.


In [7]:
def create_c1node(X_feed,y_now):
    model = Sequential()
    model.add(Dense(128,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='he_normal'))
    model.add(Dense(64,activation='relu',kernel_initializer='he_normal'))
    model.add(Dense(y_now.shape[1],activation='linear',kernel_initializer='he_normal'))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    
    return model

In [8]:
from time import time

In [9]:
chain = []

X_feed = X_train.copy()
t1 = time()

for i in range(y_train.shape[1]):
    print('Training chain node ',i)
    y_now = y_train[:,[i,]].copy()
    print('Shapes\n X = {} \n y = {}'.format(X_feed.shape,y_now.shape))
    
    node = create_c1node(X_feed,y_now)
    node.fit(X_feed,y_now,epochs=500,batch_size=50)
    print('Training of node {} complete\n'.format(i))
    chain.append(node)
    X_feed = np.append(X_feed,y_now,axis=1)
    
t2 = time()

print('Time taken ',(t2-t1))

Training chain node  0
Shapes
 X = (7842, 280) 
 y = (7842, 1)





Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Epoch 1/500
7842/7842 [==============================] - 1s - loss: 0.2484 - mean_squared_error: 0.2484     
Epoch 2/500
7842/7842 [==============================] - 0s - loss: 0.1241 - mean_squared_error: 0.1241     
Epoch 3/500
7842/7842 [==============================] - 0s - loss: 0.0916 - mean_squared_error: 0.0916     
Epoch 4/500
7842/7842 [==============================] - ETA: 0s - loss: 0.0810 - mean_squared_error: 0.08 - 0s - loss: 0.0807 - mean_squared_error: 0.0807     
Epoch 5/500
7842/7842 [==============================] - 0s - loss: 0.0795 - mean_squared_error: 0.0795     
Epoch 6/500
7842/7842 [==============================] - 0s - loss: 0.0704 - mean_squared_error: 0.0704     
Epoch 7/500
7842/7842 [==============================] - 0s - loss: 0.0649 - mean_squared_error: 0.0649    

7842/7842 [==============================] - 0s - loss: 0.0196 - mean_squared_error: 0.0196     
Epoch 61/500
7842/7842 [==============================] - 0s - loss: 0.0194 - mean_squared_error: 0.0194     
Epoch 62/500
7842/7842 [==============================] - 0s - loss: 0.0194 - mean_squared_error: 0.0194     
Epoch 63/500
7842/7842 [==============================] - 0s - loss: 0.0184 - mean_squared_error: 0.0184     
Epoch 64/500
7842/7842 [==============================] - 0s - loss: 0.0206 - mean_squared_error: 0.0206     
Epoch 65/500
7842/7842 [==============================] - 0s - loss: 0.0196 - mean_squared_error: 0.0196      ETA: 0s - loss: 0.0167 - mean
Epoch 66/500
7842/7842 [==============================] - 0s - loss: 0.0223 - mean_squared_error: 0.0223     
Epoch 67/500
7842/7842 [==============================] - 0s - loss: 0.0196 - mean_squared_error: 0.0196     
Epoch 68/500
7842/7842 [==============================] - 0s - loss: 0.0191 - mean_squared_error: 0.019

7842/7842 [==============================] - 0s - loss: 0.0077 - mean_squared_error: 0.0077     
Epoch 134/500
7842/7842 [==============================] - 0s - loss: 0.0085 - mean_squared_error: 0.0085     
Epoch 135/500
7842/7842 [==============================] - 0s - loss: 0.0084 - mean_squared_error: 0.0084     
Epoch 136/500
7842/7842 [==============================] - 0s - loss: 0.0087 - mean_squared_error: 0.0087     
Epoch 137/500
7842/7842 [==============================] - 0s - loss: 0.0087 - mean_squared_error: 0.0087     
Epoch 138/500
7842/7842 [==============================] - 0s - loss: 0.0089 - mean_squared_error: 0.0089     
Epoch 139/500
7842/7842 [==============================] - 0s - loss: 0.0105 - mean_squared_error: 0.0105     
Epoch 140/500
7842/7842 [==============================] - 0s - loss: 0.0100 - mean_squared_error: 0.0100     
Epoch 141/500
7842/7842 [==============================] - 0s - loss: 0.0087 - mean_squared_error: 0.0087     
Epoch 142/500
7

7842/7842 [==============================] - ETA: 0s - loss: 0.0045 - mean_squared_error: 0.00 - 0s - loss: 0.0045 - mean_squared_error: 0.0045     
Epoch 206/500
7842/7842 [==============================] - 0s - loss: 0.0046 - mean_squared_error: 0.0046     
Epoch 207/500
7842/7842 [==============================] - 0s - loss: 0.0046 - mean_squared_error: 0.0046     
Epoch 208/500
7842/7842 [==============================] - 0s - loss: 0.0052 - mean_squared_error: 0.0052     
Epoch 209/500
7842/7842 [==============================] - 0s - loss: 0.0049 - mean_squared_error: 0.0049     
Epoch 210/500
7842/7842 [==============================] - 0s - loss: 0.0050 - mean_squared_error: 0.0050     
Epoch 211/500
7842/7842 [==============================] - 0s - loss: 0.0061 - mean_squared_error: 0.0061     
Epoch 212/500
7842/7842 [==============================] - 0s - loss: 0.0055 - mean_squared_error: 0.0055     
Epoch 213/500
7842/7842 [==============================] - 0s - loss: 0.00

7842/7842 [==============================] - 0s - loss: 0.0045 - mean_squared_error: 0.0045     
Epoch 279/500
7842/7842 [==============================] - 0s - loss: 0.0042 - mean_squared_error: 0.0042     
Epoch 280/500
7842/7842 [==============================] - 0s - loss: 0.0042 - mean_squared_error: 0.0042     
Epoch 281/500
7842/7842 [==============================] - 0s - loss: 0.0033 - mean_squared_error: 0.0033     
Epoch 282/500
7842/7842 [==============================] - 0s - loss: 0.0029 - mean_squared_error: 0.0029     
Epoch 283/500
7842/7842 [==============================] - 0s - loss: 0.0027 - mean_squared_error: 0.0027     
Epoch 284/500
7842/7842 [==============================] - 0s - loss: 0.0024 - mean_squared_error: 0.0024     
Epoch 285/500
7842/7842 [==============================] - 0s - loss: 0.0040 - mean_squared_error: 0.0040     
Epoch 286/500
7842/7842 [==============================] - 0s - loss: 0.0037 - mean_squared_error: 0.0037     
Epoch 287/500
7

...

7842/7842 [==============================] - 0s - loss: 0.0050 - mean_squared_error: 0.0050     
Epoch 206/500
7842/7842 [==============================] - 0s - loss: 0.0045 - mean_squared_error: 0.0045     
Epoch 207/500
7842/7842 [==============================] - 0s - loss: 0.0049 - mean_squared_error: 0.0049     
Epoch 208/500
7842/7842 [==============================] - 0s - loss: 0.0047 - mean_squared_error: 0.0047     
Epoch 209/500
7842/7842 [==============================] - ETA: 0s - loss: 0.0045 - mean_squared_error: 0.00 - 0s - loss: 0.0045 - mean_squared_error: 0.0045     
Epoch 210/500
7842/7842 [==============================] - 0s - loss: 0.0050 - mean_squared_error: 0.0050     
Epoch 211/500
7842/7842 [==============================] - 0s - loss: 0.0054 - mean_squared_error: 0.0054     
Epoch 212/500
7842/7842 [==============================] - 0s - loss: 0.0056 - mean_squared_error: 0.0056     
Epoch 213/500
7842/7842 [==============================] - 0s - loss: 0.00

7842/7842 [==============================] - 0s - loss: 0.0031 - mean_squared_error: 0.0031     
Epoch 278/500
7842/7842 [==============================] - 0s - loss: 0.0039 - mean_squared_error: 0.0039     
Epoch 279/500
7842/7842 [==============================] - 0s - loss: 0.0038 - mean_squared_error: 0.0038     
Epoch 280/500
7842/7842 [==============================] - 0s - loss: 0.0036 - mean_squared_error: 0.0036     
Epoch 281/500
7842/7842 [==============================] - ETA: 0s - loss: 0.0038 - mean_squared_error: 0.00 - 0s - loss: 0.0038 - mean_squared_error: 0.0038     
Epoch 282/500
7842/7842 [==============================] - 0s - loss: 0.0032 - mean_squared_error: 0.0032     
Epoch 283/500
7842/7842 [==============================] - 0s - loss: 0.0033 - mean_squared_error: 0.0033     
Epoch 284/500
7842/7842 [==============================] - 0s - loss: 0.0030 - mean_squared_error: 0.0030     
Epoch 285/500
7842/7842 [==============================] - 0s - loss: 0.00

7842/7842 [==============================] - 0s - loss: 0.0045 - mean_squared_error: 0.0045     
Epoch 350/500
7842/7842 [==============================] - 0s - loss: 0.0047 - mean_squared_error: 0.0047     
Epoch 351/500
7842/7842 [==============================] - 0s - loss: 0.0029 - mean_squared_error: 0.0029     
Epoch 352/500
7842/7842 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021     
Epoch 353/500
7842/7842 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020     
Epoch 354/500
7842/7842 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020     
Epoch 355/500
7842/7842 [==============================] - 0s - loss: 0.0023 - mean_squared_error: 0.0023     
Epoch 356/500
7842/7842 [==============================] - 0s - loss: 0.0022 - mean_squared_error: 0.0022     
Epoch 357/500
7842/7842 [==============================] - 0s - loss: 0.0025 - mean_squared_error: 0.0025     
Epoch 358/500
7

7842/7842 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021     
Epoch 423/500
7842/7842 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020     
Epoch 424/500
7842/7842 [==============================] - 0s - loss: 0.0022 - mean_squared_error: 0.0022     
Epoch 425/500
7842/7842 [==============================] - 0s - loss: 0.0025 - mean_squared_error: 0.0025     
Epoch 426/500
7842/7842 [==============================] - 0s - loss: 0.0030 - mean_squared_error: 0.0030      ETA: 0s - loss: 0.0021 
Epoch 427/500
7842/7842 [==============================] - 0s - loss: 0.0029 - mean_squared_error: 0.0029     
Epoch 428/500
7842/7842 [==============================] - 0s - loss: 0.0025 - mean_squared_error: 0.0025     
Epoch 429/500
7842/7842 [==============================] - 0s - loss: 0.0020 - mean_squared_error: 0.0020     
Epoch 430/500
7842/7842 [==============================] - 0s - loss: 0.0019 - mean_squared_error: 0.0

7842/7842 [==============================] - 1s - loss: 0.0025 - mean_squared_error: 0.0025      ETA: 0s - loss: 0.0025 - mean_squared_error: 0.
Epoch 496/500
7842/7842 [==============================] - 0s - loss: 0.0022 - mean_squared_error: 0.0022     
Epoch 497/500
7842/7842 [==============================] - 0s - loss: 0.0021 - mean_squared_error: 0.0021     
Epoch 498/500
7842/7842 [==============================] - 0s - loss: 0.0018 - mean_squared_error: 0.0018     
Epoch 499/500
7842/7842 [==============================] - 0s - loss: 0.0018 - mean_squared_error: 0.0018     
Epoch 500/500
7842/7842 [==============================] - 0s - loss: 0.0018 - mean_squared_error: 0.0018     
Training of node 15 complete

Time taken  5767.783168315887


In [10]:
y_pred = []

X_feed = X_test.copy()
i = 0
for node in chain:
    print('getting output from chain node ',i)
    node_name = 'Node' + str(i)
    print('Shapes - \n X = {}'.format(X_feed.shape))
    output = node.predict(X_feed)
    y_pred.append(output)
    X_feed = np.append(X_feed,output,axis=1)
    i += 1

getting output from chain node  0
Shapes - 
 X = (1961, 280)
getting output from chain node  1
Shapes - 
 X = (1961, 281)
getting output from chain node  2
Shapes - 
 X = (1961, 282)
getting output from chain node  3
Shapes - 
 X = (1961, 283)
getting output from chain node  4
Shapes - 
 X = (1961, 284)
getting output from chain node  5
Shapes - 
 X = (1961, 285)
getting output from chain node  6
Shapes - 
 X = (1961, 286)
getting output from chain node  7
Shapes - 
 X = (1961, 287)
getting output from chain node  8
Shapes - 
 X = (1961, 288)
getting output from chain node  9
Shapes - 
 X = (1961, 289)
getting output from chain node  10
Shapes - 
 X = (1961, 290)
getting output from chain node  11
Shapes - 
 X = (1961, 291)
getting output from chain node  12
Shapes - 
 X = (1961, 292)
getting output from chain node  13
Shapes - 
 X = (1961, 293)
getting output from chain node  14
Shapes - 
 X = (1961, 294)
getting output from chain node  15
Shapes - 
 X = (1961, 295)


In [11]:
predictions = np.array([0])
for i in y_pred:
    x = np.array(i)
    if predictions.shape == (1,):
        predictions = x
    else:
        predictions = np.append(predictions,x,axis=1)

In [12]:
evaluate(scaler.inverse_transform(y_test),scaler.inverse_transform(np.array(predictions)))

MAE -  48.800313389343856
RMSE -  81.47770551007737
R-squared -  0.9120077269992339
Adjusted R-squared -  0.9094202625547669
